In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin

In [ ]:
def get_all_links(url, depth=0, visited={}):
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  links = soup.find_all('a')
  links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
  links = [link for link in links if link.startswith(url) or link.startswith('/')]
  links = [urljoin(url, link) for link in links if link]


  # Recursively crawl the links at the next depth level
  if depth < 2: #3
    new_links = []
    for link in links:
      # Increment the visit count for the link
      if link in visited:
        visited[link] += 1
      else:
        visited[link] = 1
      # Get the newly-crawled links and add them to the list
        new_links.extend(get_all_links(link, depth=depth+1, visited=visited))
      # Add the newly-crawled links to the original list
    links.extend(new_links)

  return visited

base_url = 'https://www.thairath.co.th'
website_dict1 = get_all_links(base_url, depth=0, visited={})
print(website_dict1)


In [ ]:
def crawl(url, depth,visited = set()):
    if depth < 2 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [link for link in links if link.startswith(url) or link.startswith('/')]
        links = [urljoin(url, link) for link in links if link]
        for link in links:
            if link not in visited:
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://www.msn.com/th-th'
website_dict2 = crawl(base_url, 0, visited=set())
print(website_dict2)


In [23]:
def crawl(url, depth,visited = set()):
    if depth < 4 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        response = requests.get(url,headers=headers)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [link for link in links if link.startswith(url) or link.startswith('/')]
        links = [urljoin(url, link) for link in links if link]
        for link in links:
            if link not in visited:
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://thairath.co.th'
website_dict2 = crawl(base_url, 0, visited=set())
print(website_dict2)


{'https://thairath.co.th/podcast/program/extratime/episode63', 'https://thairath.co.th/member/profile/following', 'https://thairath.co.th/business/investment/gold', 'https://thairath.co.th/refresh', 'https://thairath.co.th/lifestyle/life', 'https://thairath.co.th/column/newspaper/around', 'https://thairath.co.th/sport/eurofootball/premierleague/2621005', 'https://thairath.co.th/tv', 'https://thairath.co.th/lifestyle', 'https://thairath.co.th/news/politic/2620233', 'https://thairath.co.th/scoop/theissue/2561886', 'https://thairath.co.th/tv/program/Setthee_Pai_Deang', 'https://thairath.co.th/tags/พ.พาทินี', 'https://thairath.co.th/tv/program/Setthee_Pai_Deang/710265', 'https://thairath.co.th/business/investment/gold/topic', 'https://thairath.co.th/news/local/bangkok/2620225', 'https://thairath.co.th/tv/program/thairath_news_show/719005', 'https://thairath.co.th/lifestyle/caption/2206631', 'https://thairath.co.th', 'https://thairath.co.th/lifestyle/caption', 'https://thairath.co.th/news/c

In [ ]:
def crawl(url, depth,visited = set()):
    if depth < 3 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [urljoin(url, link) for link in links if link or link.startswith('/') ]
        for link in links:
            if link not in visited:
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://www.thairath.co.th'
website_dict3 = crawl(base_url, 0, visited=set())
print(website_dict2)

In [1]:
a1={'https://www.thairath.co.th/home': 1, 'https://www.thairath.co.th/tv/live': 2, 'https://www.thairath.co.th/news/local/north/2620546': 4, 'https://www.thairath.co.th/news/local/central/2620555': 4, 'https://www.thairath.co.th/sport/eurofootball/premierleague/2620385': 4, 'https://www.thairath.co.th/news/politic/2620501': 4, 'https://www.thairath.co.th/entertain/news/2620506': 4, 'https://www.thairath.co.th/scoop/interview/2620466': 4, 'https://www.thairath.co.th/spotlight': 2, 'https://www.thairath.co.th/spotlight/covid-19-vaccine': 6, 'https://www.thairath.co.th/spotlight/bualuang-printing': 4, 'https://www.thairath.co.th/spotlight/bblfinancialtree/': 4, 'https://www.thairath.co.th/spotlight/onlinelearning': 4, 'https://www.thairath.co.th/column': 2, 'https://www.thairath.co.th/news/politic/2618973': 2, 'https://www.thairath.co.th/column/newspaper/greenhead': 1, 'https://www.thairath.co.th/news/foreign/2618970': 1, 'https://www.thairath.co.th/column/newspaper/worldsky': 1, 'https://www.thairath.co.th/sport/others/2619893': 1, 'https://www.thairath.co.th/news/local/2619015': 1, 'https://www.thairath.co.th/column/newspaper/page1scoop': 1, 'https://www.thairath.co.th/news/politic/2619663': 1, 'https://www.thairath.co.th/column/newspaper/wikroh': 1, 'https://www.thairath.co.th/member/profile/following': 2, 'https://www.thairath.co.th/scoop': 2, 'https://www.thairath.co.th/scoop/theissue/2620528': 2, 'https://www.thairath.co.th/scoop/theissue': 3, 'https://www.thairath.co.th/scoop/flashback/2620025': 2, 'https://www.thairath.co.th/scoop/flashback': 1, 'https://www.thairath.co.th/scoop/theissue/2619623': 2, 'https://www.thairath.co.th/scoop/theissue/2619605': 2, 'https://www.thairath.co.th/scoop/culture/2619553': 2, 'https://www.thairath.co.th/scoop/culture': 1, 'https://www.thairath.co.th/worldcupactivity': 3, 'https://www.thairath.co.th/campaign/vote': 2, 'https://www.thairath.co.th/refresh': 3, 'https://www.thairath.co.th/news/politic/2607053': 2, 'https://www.thairath.co.th/lifestyle/life/2529290': 2, 'https://www.thairath.co.th/scoop/theissue/2561886': 2, 'https://www.thairath.co.th/video': 2, 'https://www.thairath.co.th/podcast': 2, 'https://www.thairath.co.th/podcast/program/ghoststation/episode72': 2, 'https://www.thairath.co.th/podcast/program/Howtolove/episode141': 2, 'https://www.thairath.co.th/lifestyle': 2, 'https://www.thairath.co.th/news/local/2619423': 2, 'https://www.thairath.co.th/news/local': 2, 'https://www.thairath.co.th/news/local/2619108': 2, 'https://www.thairath.co.th/horoscope/dailyhoro/2614590': 2, 'https://www.thairath.co.th/tags/พ.พาทินี': 1, 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033': 2, 'https://www.thairath.co.th/horoscope/horoscope-seer': 1, 'https://www.thairath.co.th/lifestyle/life/2620008': 2, 'https://www.thairath.co.th/lifestyle/life': 3, 'https://www.thairath.co.th/lifestyle/life/2620181': 2, 'https://www.thairath.co.th/lifestyle/life/2619120': 2, 'https://www.thairath.co.th/lifestyle/food/2619526': 2, 'https://www.thairath.co.th/lifestyle/food': 1, 'https://www.thairath.co.th/tv': 1, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew': 1, 'https://www.thairath.co.th/tv/program/talon_khao': 1, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath': 1, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath': 1, 'https://www.thairath.co.th/tv/program/khao_sai_khai': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show': 5, 'https://www.thairath.co.th/tv/program': 1, 'https://www.thairath.co.th/tv/watch/news': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/718740': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/718735': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/718725': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/718721': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/718730': 2, 'https://www.thairath.co.th/tv/program/Poet_Pak_Kap_Phak_phum/718710': 2, 'https://www.thairath.co.th/tv/program/Poet_Pak_Kap_Phak_phum': 1, 'https://www.thairath.co.th/tv/program/buntoeng_thairath/718638': 2, 'https://www.thairath.co.th/tv/program/buntoeng_thairath': 1, 'https://www.thairath.co.th/tv/watch': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/715568': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang': 4, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330': 2, 'https://www.thairath.co.th/lottery': 3, 'https://www.thairath.co.th/business/investment/gold/history': 2, 'https://www.thairath.co.th/business/investment/gold': 1, 'https://www.thairath.co.th/business/investment/gold/topic': 1, 'https://www.thairath.co.th/multimedia/gallery': 2, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218': 6, 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208': 6, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860': 6, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851': 6, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34310': 4, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225': 4, 'https://www.thairath.co.th/ads/thairathOnline': 1}
b2={'https://www.thairath.co.th/scoop/theissue/2619605', 'https://www.thairath.co.th/news/local/north/2620546', 'https://www.thairath.co.th/sport/eurofootball/premierleague/2620385', 'https://www.thairath.co.th/news/local/2619015', 'https://www.thairath.co.th/spotlight/bualuang-printing', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101', 'https://www.thairath.co.th/video', 'https://www.thairath.co.th/tv/program/khao_sai_khai', 'https://www.thairath.co.th/tv/watch/news', 'https://www.thairath.co.th/podcast/program/Howtolove/episode141', 'https://www.thairath.co.th/scoop/flashback/2620025', 'https://www.thairath.co.th/lifestyle/life/2529290', 'https://www.thairath.co.th', 'https://www.thairath.co.th/entertain/news/2620506', 'https://www.thairath.co.th/tv', 'https://www.thairath.co.th/column/newspaper/page1scoop', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/718638', 'https://www.thairath.co.th/column/newspaper/wikroh', 'https://www.thairath.co.th/news/politic/2607053', 'https://www.thairath.co.th/news/local', 'https://www.thairath.co.th/news/local/central/2620555', 'https://www.thairath.co.th/spotlight', 'https://www.thairath.co.th/spotlight/bblfinancialtree/', 'https://www.thairath.co.th/business/investment/gold/topic', 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851', 'https://www.thairath.co.th/tv/watch', 'https://www.thairath.co.th/podcast', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218', 'https://www.thairath.co.th/news/politic/2619663', 'https://www.thairath.co.th/horoscope/horoscope-seer', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225', 'https://www.thairath.co.th/lottery', 'https://www.thairath.co.th/tv/program/Poet_Pak_Kap_Phak_phum', 'https://www.thairath.co.th/column/newspaper/greenhead', 'https://www.thairath.co.th/scoop/theissue/2620528', 'https://www.thairath.co.th/tv/program/thairath_news_show/718740', 'https://www.thairath.co.th/column/newspaper/worldsky', 'https://www.thairath.co.th/podcast/program/ghoststation/episode72', 'https://www.thairath.co.th/tv/program/thairath_news_show/718721', 'https://www.thairath.co.th/campaign/vote', 'https://www.thairath.co.th/sport/others/2619893', 'https://www.thairath.co.th/business/investment/gold/history', 'https://www.thairath.co.th/spotlight/covid-19-vaccine', 'https://www.thairath.co.th/lifestyle/life/2619120', 'https://www.thairath.co.th/column', 'https://www.thairath.co.th/lifestyle', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang', 'https://www.thairath.co.th/scoop/culture/2619553', 'https://www.thairath.co.th/lifestyle/food/2619526', 'https://www.thairath.co.th/scoop/theissue/2619623', 'https://www.thairath.co.th/tv/program/khao_sai_khai/718730', 'https://www.thairath.co.th/business/investment/gold', 'https://www.thairath.co.th/scoop/theissue', 'https://www.thairath.co.th/tv/program/talon_khao', 'https://www.thairath.co.th/news/politic/2618973', 'https://www.thairath.co.th/news/local/2619108', 'https://www.thairath.co.th/horoscope/dailyhoro/2614590', 'https://www.thairath.co.th/tv/program/thairath_news_show/718725', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860', 'https://www.thairath.co.th/lifestyle/life', 'https://www.thairath.co.th/news/foreign/2618970', 'https://www.thairath.co.th/tv/program/buntoeng_thairath', 'https://www.thairath.co.th/tv/program/khoa_yen_thairath', 'https://www.thairath.co.th/member/profile/following', 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033', 'https://www.thairath.co.th/lifestyle/life/2620008', 'https://www.thairath.co.th/tv/live', 'https://www.thairath.co.th/scoop/culture', 'https://www.thairath.co.th/lifestyle/life/2620181', 'https://www.thairath.co.th/refresh', 'https://www.thairath.co.th/tags/พ.พาทินี', 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew', 'https://www.thairath.co.th/worldcupactivity', 'https://www.thairath.co.th/tv/program/khao_tieng_thairath', 'https://www.thairath.co.th/home', 'https://www.thairath.co.th/lifestyle/food', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34310', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/715568', 'https://www.thairath.co.th/ads/thairathOnline', 'https://www.thairath.co.th/scoop/theissue/2561886', 'https://www.thairath.co.th/tv/program', 'https://www.thairath.co.th/tv/program/Poet_Pak_Kap_Phak_phum/718710', 'https://www.thairath.co.th/tv/program/thairath_news_show', 'https://www.thairath.co.th/scoop', 'https://www.thairath.co.th/news/local/2619423', 'https://www.thairath.co.th/news/politic/2620501', 'https://www.thairath.co.th/multimedia/gallery', 'https://www.thairath.co.th/tv/program/thairath_news_show/718735', 'https://www.thairath.co.th/spotlight/onlinelearning', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265', 'https://www.thairath.co.th/scoop/interview/2620466', 'https://www.thairath.co.th/scoop/flashback'}
a=website_dict2
b=website_dict3
print (len(a))
print(len(b))

for i in a:
    if i not in b :
        print((i))

NameError: name 'website_dict2' is not defined

In [ ]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  
  if  title_tag == None:
    title_tag = summarize_article[0]
    
  return p_tag, title_tag, keyword

def insert_data_to_db(website, p_tag, title, keyword, website_dict):
  
  for i in keyword:
    conn.execute("INSERT INTO DATA (WEBSITE, BODY, TITLE, KEYWORD, WORD_FREQUENCY ,REF) VALUES (?, ?, ?, ?, ?, ?)", (website, p_tag, title, i, keyword[i],website_dict[website]))
    conn.commit()
    print(f'For website {website}')
    print(f'For the p tags is:{p_tag}') 
    print(f'For the title tag is: {title}')
    print(f'For the keyword is:{i}')
    print(f'Forthe word frequency is:{keyword[i]}')
    print(f'For the ref is:{website_dict[website]}')
 

# Iterate through the websites
conn = sqlite3.connect('scraped_data.db')
conn.execute('''CREATE TABLE DATA
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             WEBSITE STRING NOT NULL,
             BODY TEXT NOT NULL,
             TITLE TEXT NOT NULL,
             KEYWORD TEXT NOT NULL,
             WORD_FREQUENCY INT NOT NULL,
             REF INT NOT NULL);''')
for website in website_dict1.keys():
    if 'news' in website:
        p_tag, title, keyword = scrape_tags(website)
        insert_data_to_db(website, p_tag, title, keyword, website_dict1)

conn.close()






In [ ]:
keyword = {}
keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
print(keyword)